<a href="https://colab.research.google.com/github/Azure06072005/DS317-Data-Mining-for-Enterprise/blob/main/merge_reply_user_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import necessary libraries


In [2]:
import pyspark
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt
from os import truncate

warnings.filterwarnings("ignore")

spark = SparkSession.builder \
  .appName("reply_user_video_analysis") \
  .master("local[*]") \
  .config("spark.some.config.option", "some-value") \
  .getOrCreate()

In [3]:
BASE_DIR = '/content/drive/MyDrive/DS317.Q11 nhóm 8/DataMOOCDS317'
MY_TEST_DIR = '/content/drive/MyDrive/DS317.Q11 nhóm 8/Bài tập trên lớp/Bài tập phân tích Dataset/Data/test'
df_user_course_path = '/content/drive/MyDrive/DS317.Q11 nhóm 8/DataMOOCDS317/user_course_final_ver3/user_course_v4.csv'

# User-video

In [ ]:
df_user_video = spark.read.json(BASE_DIR + '/user-video.json')


In [ ]:
df_user_video = spark.read.json(BASE_DIR + '/user-video.json')
# Explode the outer array in 'seq' to get each video sequence as a row
df_user_video = df_user_video.select("user_id", explode("seq").alias("video_sequence"))

# Select the video_id and explode the 'segment' array within each video sequence
df_user_video = df_user_video.select(
    "user_id",
    df_user_video["video_sequence"]["video_id"].alias("video_id"),
    explode(df_user_video["video_sequence"]["segment"]).alias("segment_details")
)

# Select the individul fields from the 'segment_details' struct
df_user_video = df_user_video.select(
    "user_id",
    "video_id",
    df_user_video["segment_details"]["start_point"].alias("start_point"),
    df_user_video["segment_details"]["end_point"].alias("end_point"),
    df_user_video["segment_details"]["speed"].alias("speed"),
    df_user_video["segment_details"]["local_start_time"].alias("local_start_time")
)

df_user_video = df_user_video.withColumn("watched_duration", round(col('end_point') - col('start_point')))
df_user_video = df_user_video.select("user_id", "video_id", "speed", "watched_duration")

In [ ]:
df_user_video.show()

In [ ]:
df_user_video_grouped = (
    df_user_video
    .groupBy("user_id")
    .agg(
        (F.sum(F.col("watched_duration") * F.col("speed")) / F.sum(F.col("watched_duration"))).alias("avg_speed"),
        F.sum("watched_duration").alias("total_watched_duration"),
        F.collect_list("video_id").alias("video_list")
    )
)

In [ ]:
df_user_video_grouped.show()

In [ ]:
df_user_course = spark.read.csv(df_user_course_path, header=True, inferSchema=True)
df_uc_uv = df_user_course.join(df_user_video_grouped, on=['user_id'], how='left')
df_uc_uv.show(truncate=False)

In [ ]:
df_uc_uv.count()

In [ ]:
df_uc_uv.select([count(when(col(c).isNull(), c)).alias(c) for c in df_uc_uv.columns]).show()

In [ ]:
df_video = spark.read.json(BASE_DIR + '/video.json')
df_video.show(5,truncate=True)

In [ ]:
from pyspark.sql.functions import array_max
df_video = df_video.withColumn('video_duration', array_max('end'))

In [ ]:
df_video_id_ccid = spark.read.text(BASE_DIR + '/video_id-ccid.txt')

In [ ]:
from pyspark.sql.functions import split

# Tách cột value thành 2 cột video_id và ccid
df_video_id_ccid = df_video_id_ccid.withColumn('video_id', split(df_video_id_ccid['value'], '\t').getItem(0)) \
           .withColumn('ccid', split(df_video_id_ccid['value'], '\t').getItem(1))
df_video_id_ccid = df_video_id_ccid.select('ccid', 'video_id')
df_video_id_ccid.show(truncate=False)


In [ ]:
df_video = df_video.join(df_video_id_ccid, on=['ccid'], how='left')
df_video.show(5)

+--------------------+--------------------+--------------+--------------------+---------------------------------+--------------+--------+--------+--------+---------+
|                ccid|                 end|          name|               start|                             text|video_duration|video_id|video_id|video_id| video_id|
+--------------------+--------------------+--------------+--------------------+---------------------------------+--------------+--------+--------+--------+---------+
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448343|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448628|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84, 23....|[各位同学 大家好, 我是来自中国...|       492.315|V_448343|V_448343|V_448343| V_448757|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|家禽的消化系统|[19.2, 20.84,

In [ ]:
df_video = df_video.select("video_duration", "video_id")

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `video_id` is ambiguous, could be: [`video_id`, `video_id`, `video_id`].

In [ ]:
df_course = spark.read.json(BASE_DIR + '/course.json')
df_course.show(5,truncate=True)

In [ ]:
df_video.printSchema()

In [ ]:
df_video.show(5)

In [ ]:
df_video_duration = df_video.select('video_id', 'duration')
df_video_duration.show(5)

In [ ]:
df_user_video.show(5)

In [ ]:
df_user_video = df_user_video.join(df_video_duration, on=['video_id'], how='left')
df_user_video.show(5)

In [ ]:
df_uc_uv = df_uc_uv.join(df_video_duration, on = ["video_id"], how = 'left')
df_uc_uv.show(5)

In [ ]:
from pyspark.sql.functions import col, isnan, when, count
df_uc_uv.count()
df_uc_uv.select([count(when(col(c).isNull(), c)).alias(c) for c in df_uc_uv.columns]).show()

In [ ]:
df_user_course.show(5)

In [ ]:
# df_uc_uv.write.mode('overwrite').parquet(MY_TEST_DIR)